# 💥 plongement de mot

3 étapes :
- charger un plongement Word2Vec précalculé 
- écrire une fonction qui trouve les voisins d'un mot = ses mots les plus proches
- explorer les analogies
  

    
___



#### charger le word2vec précalculé 
  - chercher frwiki-50k_100d.txt sur google drive et le mettre à côté de votre notebook.
  - c'est une réduction à 50k types les plus fréquents du word2vec complet https://wikipedia2vec.github.io/wikipedia2vec/pretrained/ 
  - ouvrir le fichier et comprendre le format
  - 🚧 écrire une fonction `read_vectors` qui donne une matrice numpy `embeddings` avec une ligne par type (~ un mot), chaque ligne représente le mot. `embeddings[i,:]` correspond à un type.
  

    

In [ ]:
import numpy as np

In [ ]:
def read_vectors(infile, vocab_size=50000, dimension=100):
    """
    fonction qui lit une sauvegarde d'un plongement
    retourne 
    - un dictionnaire word type -> index
    - le dictionnaire inverse
    - une matrice contenant un vecteur de mot par word type
    """  
    
    embeddings = np.zeros((vocab_size, dimension))
    w2i, i2w = {}, {}
    i = 0
    for line in open(infile, encoding='utf8'): # iso-88..."
        splitted = line.split()
        w2i[splitted[0]] = i
        i2w[i] = splitted[0]
        embeddings[i] = np.array(splitted[1:], dtype=float)
        i+=1
    return w2i, i2w, embeddings

w2i, i2w, embeddings = read_vectors('/home/flandes/data/frwiki-50k_100d.txt')
print(len(w2i),len(i2w),embeddings.shape)

In [ ]:
i2w[40000]

In [ ]:
embeddings[40000]

In [ ]:
w2i, i2w, embeddings

### Objectifs
On veut faire une fonction python qui pour un mot donné retourne les K plus proches voisins (sémantiques)
Pour ce faire, il faudra 
1. calculer la distance cosinus entre ?? et ??
2. ordoner les distances par ordres decroissants


#### 1. Calcul de la distance cosinus
**Astuce**, on peut faire ceci en deux sous-étapes : 
- 🚧 normaliser les embeddings (par rapport aux lignes, càd chaque vecteur de mot à une norme unitaire)
- 🚧 calculer produit scalaire entre l'embedding du mot source avec l'espace d'embedding entier

In [ ]:
normed_embeddings = embeddings/(np.linalg.norm(embeddings, axis=1)).reshape(-1, 1)

**⚠ Achtung !!!** : Pour simplifier les calculs par la suite, on basera nos calculs sur la matrice `normed_embeddings`. De ce fait, on n'aura pas besoin de diviser le produit scalaire des deux vecteurs par leurs normes (la similarité cosinus de deux vecteurs unitaires est juste le produit scalaire des vecteurs).
Si vous souhaitez effectuer un calcul de similarité basé sur une autre mesure (distance euclidienne ?), alors vous devrez baser vos calculs sur la matrice `embeddings` brute (qui n'est pas normalisée !)

In [ ]:
def find_closest_words_from_vector(vector, K=10):
    """Pour un vecteur donné, trouve les K plus proches voisins sémantiques"""

#     TODO
    
    return closest_words

In [ ]:
def find_closest_words_from_word(word, K=10):
    """Pour un mot 'word' donné, trouve les K plus proches voisins sémantiques"""

#     TODO 

    return closest_words

In [ ]:
def pretty_show(closest_words):
    """prettify the print for the closest words"""
    for word, similarity in closest_words:
        print(word.ljust(15, ' '), similarity.round(3))
    print("___\n")

In [ ]:
# Creative time ! Testons notre fonction sur certains termes
pretty_show(find_closest_words_from_word("français")) # notez comment un plongement peut être utilisé pour normaliser l'orthographe. notez aussi que l'allemand est plus proche du français que marseillais :)
pretty_show(find_closest_words_from_word("fusée"))
pretty_show(find_closest_words_from_word("purée")) # c'est entraîné sur Wikipédia. --> peu de jurons...
pretty_show(find_closest_words_from_word("mince"))

### analogies
finalement les jolies analogies dans les plongements !

In [ ]:
def find_analogies(wpos1, wpos2, wneg):
    """
    wpos1: le premier mot positif, wpos2: le 2e mot positif, wneg: le mot négatif
    trouve l'analogie 'analog' tel que <'analog' est à 'wpos2' ce que 'wneg' est à 'wpos1'>
    Par exemple : <"reine" est à "femme" ce que "roi" est à "homme">
    """
    # additionner les vecteurs positifs, substraire le vecteur négatif :
    vector_analog  = normed_embeddings[w2i[wpos1]] + normed_embeddings[w2i[wpos2]] - normed_embeddings[w2i[wneg]]
    return find_closest_words_from_vector(vector_analog)

In [ ]:
pretty_show(find_analogies("roi", "femme", "homme"))
pretty_show(find_analogies("infirmière", "homme", "femme")) # attention au biais. 
# 🚧 todo : trouvez d'autres exemples "douteux"